In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-258698
Azure region: eastus2
Subscription id: a0a76bad-11a1-4a2d-9887-97a29122c8ed
Resource group: aml-quickstarts-258698


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "auto-ml"
vm_size = "Standard_D2_V2"
max_nodes = 4

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    cluster_compute = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,max_nodes=max_nodes)
    cluster_compute = ComputeTarget.create(ws, cluster_name, compute_config)

cluster_compute.wait_for_completion(show_output=True)
print(cluster_compute.get_status())


Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling, BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
# ps = RandomParameterSampling(
#     {
#         '--C': choice(0.01,0.1,1.0,10.0),
#         '--max_iter': choice(20 ,50, 100, 120, 150, 200, 220)
#     }
# )

ps = BayesianParameterSampling(
    {
        '--C': uniform(0.01, 1000),
        '--max_iter': choice(20 ,50, 100, 120, 150, 200, 220, 250, 300, 320, 350, 400)
    }
)

# Specify a Policy
policy = BanditPolicy(
    evaluation_interval=2,
    slack_factor=0.1
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory="./",
    compute_target=cluster_compute,
    script="train.py",
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=40,
    max_concurrent_runs=4
)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
run_exp = exp.submit(config=hyperdrive_config)
run_exp.wait_for_completion(show_output=True)

RunId: HD_2368ecef-c116-4c5b-a16d-96c3e21e30e2
Web View: https://ml.azure.com/runs/HD_2368ecef-c116-4c5b-a16d-96c3e21e30e2?wsid=/subscriptions/a0a76bad-11a1-4a2d-9887-97a29122c8ed/resourcegroups/aml-quickstarts-258698/workspaces/quick-starts-ws-258698&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-05-05T04:24:31.515833][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-05-05T04:24:32.0088314Z][SCHEDULER][INFO]Scheduling job, id='HD_2368ecef-c116-4c5b-a16d-96c3e21e30e2_0' 
[2024-05-05T04:24:32.1355321Z][SCHEDULER][INFO]Scheduling job, id='HD_2368ecef-c116-4c5b-a16d-96c3e21e30e2_1' 
[2024-05-05T04:24:32.2602277Z][SCHEDULER][INFO]Scheduling job, id='HD_2368ecef-c116-4c5b-a16d-96c3e21e30e2_2' 
[2024-05-05T04:24:32.326873][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2024-05-05T04:24:32.3652667Z][SCHEDULER][INFO]Scheduling job, id='HD_2368ecef-c116-4c5b-a16d-96c3e21e30

{'runId': 'HD_2368ecef-c116-4c5b-a16d-96c3e21e30e2',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2024-05-05T04:24:30.755025Z',
 'endTimeUtc': '2024-05-05T04:55:12.50993Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '89c1a897-e1c4-476d-ad77-1ef818e173cf',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': 'infinite_space_size',
  'best_child_run_id': 'HD_2368ecef-c116-4c5b-a16d-96c3e21e30e2_22',
  'score': '0.9110773899848256',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_2368ecef-c116-4c5b-a16d-96c3e21e30e2_22'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues'

In [5]:
# import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = run_exp.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print("---------------------------------------------------------------------------")
print(f'Run Id: {best_run.id}')
print(f'Accuracy: {best_run_metrics["Accuracy"]}')
print(f'Regularization Strength: {best_run_metrics["Regularization Strength:"]}')
print(f'Max iterations: {best_run_metrics["Max iterations:"]}')
print("---------------------------------------------------------------------------")

---------------------------------------------------------------------------
Run Id: HD_2368ecef-c116-4c5b-a16d-96c3e21e30e2_22
Accuracy: 0.9110773899848255
Regularization Strength: 10.651386568041366
Max iterations: 220
---------------------------------------------------------------------------


In [6]:
print("the model is saved in the outputs directory")
os.makedirs("./outputs", exist_ok = True)
best_run.download_file("outputs/model.pkl", "outputs/manual_pipeline_model.pkl")
print("successful")

the model is saved in the outputs directory
successful


In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(path)

The remote run of Azure AutoML requires the type of data attached to training data to be TabularDatasetFactory class. Therefore, I will store the cleaned data and upload it to the default datastore, and using TabularDatasetFactory to get the train_clean_data and test_clean_data from the datastore

In [9]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
data = x.join(y)
train, test = train_test_split(data,train_size=0.3,random_state=1)
train.to_csv('training/train_clean_data.csv')
test.to_csv('training/test_clean_data.csv')

In [10]:
automl_exp = Experiment(workspace=ws,name='automl-exp')
datastore = ws.get_default_datastore()
datastore.upload(src_dir='training/',target_path='data/')
train_clean_data = TabularDatasetFactory.from_delimited_files(path=[(datastore,('data/train_clean_data.csv'))])
test_clean_data = TabularDatasetFactory.from_delimited_files(path=[(datastore,('data/test_clean_data.csv'))])

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 2 files
Uploading training/test_clean_data.csv
Uploaded training/test_clean_data.csv, 1 files out of an estimated total of 2
Uploading training/train_clean_data.csv
Uploaded training/train_clean_data.csv, 2 files out of an estimated total of 2
Uploaded 2 files


In [13]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    compute_target=cluster_compute,
    max_concurrent_iterations=4,
    training_data=train_clean_data,
    test_data=test_clean_data,
    label_column_name='y',
    n_cross_validations=3
)

In [14]:
# Submit your automl run

### YOUR CODE HERE ###
run_automl_exp = automl_exp.submit(config=automl_config)
run_automl_exp.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_56cf6df5-73f8-4076-a781-d1475c8cb408,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_56cf6df5-73f8-4076-a781-d1475c8cb408,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

{'runId': 'AutoML_56cf6df5-73f8-4076-a781-d1475c8cb408',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2024-05-05T05:03:17.331197Z',
 'endTimeUtc': '2024-05-05T05:31:36.832853Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'auto-ml',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-exp","subscription_id":"a0a76bad-11a1-4a2d-9887-97a29122c8ed","resource_group":"aml-quickstarts-258698","workspace_name":"quick-starts-ws-258698","region":"eastus2","compute_target":"auto-ml","spark_service":null,"azu

In [15]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = run_automl_exp.get_output()
best_run_metrics = best_run.get_metrics()
best_run

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_56cf6df5-73f8-4076-a781-d1475c8cb408_36,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [18]:
print("---------------------------------------------------------------------------")
print(f'Run Id: {best_run.id}')
print(f'Accuracy: {best_run_metrics["accuracy"]}')
print("---------------------------------------------------------------------------")
print("the model is saved in the outputs directory")
os.makedirs("./outputs", exist_ok = True)
best_run.download_file("outputs/model.pkl", "outputs/automl_model.pkl")
print("successful")

---------------------------------------------------------------------------
Run Id: AutoML_56cf6df5-73f8-4076-a781-d1475c8cb408_36
Accuracy: 0.9149215983813859
---------------------------------------------------------------------------
the model is saved in the outputs directory
successful


In [ ]:
cluster_compute.delete()